In [ ]:
import numpy as np
import pandas as pd

# Preparing state machine

In [ ]:
class SM:
  def __init__(self, Σ, S, S0, Sf):
    self.Σ = set(Σ)
    self.S = set(S)
    self.S0 = set(S0)
    self.Sf = set(Sf)

    if (not (self.S0 <= self.S and self.Sf <= self.S)):
      raise ValueError("S0 and Sf should be subsets of S")  
    self.δ = pd.DataFrame(columns=np.unique(list(Σ)), index=np.unique(list(S))).applymap(lambda x: set() if pd.isna(x) else x)

  def __repr__(self):
        return repr(self.δ)

  def init(self):
    for state in self.S:
      for signal in self.Σ:
        while(True):
          toStates = set(input(f'({state};{signal}) ==> ').split())

          if (len(toStates) == 0):
            break;
          elif (any(toState not in self.S for toState in toStates)):
            print("Found non existing state in user input.")
          else:
            self.δ.loc[state, signal] = toStates
            break

  def checkPathFrom(self, path, state):
    if len(path) == 0:
      return state in self.Sf
    
    signal, *tail = path
    reachable = self.δ.at[state, signal]
    return any(result == True for result in [self.checkPathFrom(tail, next) for next in reachable])
      

  def checkPath(self, path):
    return any(result == True for result in [self.checkPathFrom(path, beginning) for beginning in self.S0])

# User input

In [ ]:
sm = SM(Σ=['ε', 'a', 'b'], S=['q0', 'q1', 'qf'], S0=['q0'], Sf=['qf'])
sm.init()
sm

(q1;b) ==> q1
(q1;ε) ==> 
(q1;a) ==> q0 qf
(qf;b) ==> 
(qf;ε) ==> q1
(qf;a) ==> 
(q0;b) ==> qf
(q0;ε) ==> q1
(q0;a) ==> 


           a     b     ε
q0        {}  {qf}  {q1}
q1  {qf, q0}  {q1}    {}
qf        {}    {}  {q1}

In [ ]:
# Determination Subject
sm = SM(Σ=['ε', 'a', 'b'], S=['q0', 'q1', 'qf'], S0=['q0'], Sf=['qf'])

sm.δ.at['q0', 'ε'] = {'q1'}
sm.δ.at['q0', 'b'] = {'qf'}
sm.δ.at['q1', 'a'] = {'q0', 'qf'}
sm.δ.at['q1', 'b'] = {'q1'}
sm.δ.at['qf', 'ε'] = {'q1'}

sm

           a     b     ε
q0        {}  {qf}  {q1}
q1  {qf, q0}  {q1}    {}
qf        {}    {}  {q1}

In [ ]:
# Minimization Subject
sm = SM(Σ=['a', 'b'], S=['q0', 'q1', 'q2', 'q3', 'q4', 'q5'], S0=['q0'], Sf=['q3', 'q4', 'q5'])

sm.δ.at['q0', 'a'] = {'q1'}
sm.δ.at['q0', 'b'] = {'q2'}
sm.δ.at['q1', 'a'] = {'q2'}
sm.δ.at['q1', 'b'] = {'q3'}
sm.δ.at['q2', 'a'] = {'q1'}
sm.δ.at['q2', 'b'] = {'q4'}
sm.δ.at['q3', 'a'] = {'q5'}
sm.δ.at['q3', 'b'] = {'q4'}
sm.δ.at['q4', 'a'] = {'q3'}
sm.δ.at['q4', 'b'] = {'q4'}
sm.δ.at['q5', 'a'] = {'q5'}
sm.δ.at['q5', 'b'] = {'q4'}

sm

       a     b
q0  {q1}  {q2}
q1  {q2}  {q3}
q2  {q1}  {q4}
q3  {q5}  {q4}
q4  {q3}  {q4}
q5  {q5}  {q4}

# Epsilon transitions

In [ ]:
def getReachable(sm, state, visited=set(), signal=None):
  epsilonReachable = set()
  epsilon = sm.δ.at[state, 'ε']
  for next in [next for next in epsilon if next not in visited]:
    epsilonReachable.add(next)
    epsilonReachable.update(getReachable(sm, next, visited.union(epsilonReachable), signal))

  reachable = set()
  if signal != None:
    nextReachable = sm.δ.at[state, signal]
    for next in [next for next in nextReachable if next not in visited]:
      reachable.add(next)
      reachable.update(getReachable(sm, next, visited.union(reachable)))
  else:
    reachable.update(epsilonReachable)

  return reachable


def getEpsilonClosures(sm):
  ec = {}
  for state in sm.S:
    queue = [state]
    visited = {state}

    reachable = {state}
    while (len(queue) != 0):
      fromState = queue.pop()
      newReachable = sm.δ.at[fromState, 'ε']
      toStates = [state for state in newReachable if state not in visited]

      reachable.update(toStates)
      queue = queue + toStates
    
    ec[state] = reachable
  
  return ec


def removeEpsilonTransitions(sm):
  epsilonClosures = getEpsilonClosures(sm)
  reachableFromStartStates = {state for s0 in sm.S0 for state in epsilonClosures[s0]}

  startClosures = {}
  otherClosures = {}
  finalClosures = {}
  for state, closure in epsilonClosures.items():
    lastClassified = None
    if closure <= reachableFromStartStates:
      key = f'S0{len(startClosures)}({state})'
      startClosures[key] = (state, closure)
      lastClassified = key
    if any(reachable in sm.Sf for reachable in closure):
      key = f'Sf{len(finalClosures)}({state})'
      if lastClassified is not None:
        key = lastClassified
      finalClosures[key] = (state, closure)
      lastClassified = key
    if lastClassified is None:
      otherClosures[f'S{len(otherClosures) + 1}({state})'] = (state, closure)
  
  startStates = set(startClosures.keys())
  otherStates = set(otherClosures.keys())
  finalStates = set(finalClosures.keys())
  epsilonless_Σ = [signal for signal in sm.Σ if signal != 'ε']
  cleanSM = SM(epsilonless_Σ, startStates.union(otherStates).union(finalStates), startStates, finalStates)

  allClosures = {}
  allClosures.update(startClosures)
  allClosures.update(otherClosures)
  allClosures.update(finalClosures)

  for signal in epsilonless_Σ:
    for state in cleanSM.S:
      closure = allClosures[state][1]
      reachable = set()
      for oldState in closure:
        reachable.update(getReachable(sm, oldState, signal=signal))
      newStates = {newState for newState, (_, closure) in allClosures.items() if closure <= reachable}
      cleanSM.δ.at[state, signal] = newStates

  return cleanSM, epsilonClosures

# Determinization

In [ ]:
def determine(sm):
  if 'ε' in sm.Σ:
    sm, _ = removeEpsilonTransitions(sm)

  mappings = {'P0': set(sm.S0)}
  nonDetermined = ['P0']

  determined = SM(sm.Σ, nonDetermined, nonDetermined, [])
  while (len(nonDetermined) != 0):
    toDetermine = nonDetermined.pop()
    for signal in determined.Σ:
      reachable = set()
      for state in mappings[toDetermine]:
        reachable.update(sm.δ.at[state,signal])

      if len(reachable) != 0:
        mapping = None
        for key, value in mappings.items():
          if reachable == value:
            mapping = key
            break
        
        if not mapping:
          key = f'P{len(mappings)}'
          mappings[key] = reachable
          mapping = key
          nonDetermined.append(mapping)
          row = pd.DataFrame(columns=determined.δ.columns, index=[mapping])
          determined.δ = pd.concat([determined.δ, row])

        destination = {mapping}
      else:
        destination = {}

      determined.δ.at[toDetermine, signal] = destination

  determined.S = set(mappings.keys())
  determined.Sf = {state for state, mapping in mappings.items() if any(oldState in sm.Sf for oldState in mapping)}
  return determined, mappings

# Minimization

In [ ]:
def isDetermined(sm):
  return all(len(way) <= 1 for way in sm.δ.to_numpy().flatten())


def getAllReachable(sm):
  reachable = set(sm.S0)
  newStates = set(sm.S0)

  while (newStates != set()):
    temp = set()
    for state in newStates:
        for signal in sm.Σ:
            temp.update(sm.δ.loc[state, signal])
    newStates = temp - reachable
    reachable.update(newStates)
    
  return reachable


def removeUnreachable(sm):
  reachable = getAllReachable(sm)
  cleanSM = SM(sm.Σ, reachable, sm.S0.intersection(reachable), sm.Sf.intersection(reachable))

  for signal in sm.Σ:
    for state in reachable:
      cleanSM.δ.loc[state, signal] = {state for state in sm.δ.loc[state, signal] if state in reachable}
    
  return cleanSM


def minimize(sm):
  if not isDetermined:
    sm = determine(sm)
  sm = removeUnreachable(sm)

  classes = getClasses(sm)
  states = set(classes.keys())
  min = SM(sm.Σ, states, {'M0'}, {state for state in states if state.startswith('Mf')})
  
  for fromClass in min.S:
    for signal in min.Σ:
      if min.δ.loc[fromClass, signal] != set():
        continue
      for toClass, states in classes.items():
        found = False
        for state in classes[fromClass]:
          if len(sm.δ.loc[state, signal]) == 0:
            continue
          oldDestination = next(iter(sm.δ.loc[state, signal]))
          if oldDestination in states:
            min.δ.loc[fromClass, signal] = toClass
            found = True
            break
        if found:
          break
  
  return min, classes


def organize(classes, sm):
  organized = {}
  i = 1
  f = 0
  for clazz in classes:
    if any(state in sm.S0 for state in clazz):
      key = f'M0'
    elif any(state in sm.Sf for state in clazz):
      key = f'Mf{f}'
      f = f + 1
    else:
      key = f'M{i}'
      i = i + 1
    organized[key] = clazz
  return organized


def getClasses(sm):
  prevXi = [sm.Sf, sm.S - sm.Sf]
  while True:  
    newXi = []
    for currentClass in prevXi:
      classDf = sm.δ.loc[list(currentClass)]
      if len(currentClass) != 1:
        for signal in sm.Σ:
          classSignalDf = classDf[signal]
          
          subClasses = []
          for state in classDf.index.values:
            if len(classSignalDf[state]) == 0:
              subClasses.append(prevXi.index(currentClass))
              continue
            for i, clazz in enumerate(prevXi):
              if next(iter(classSignalDf[state])) in clazz:
                subClasses.append(i)
                break
          if len(set(subClasses)) == 1:
            continue
          
          subClasses = map(lambda x: x.to_numpy(), list(classSignalDf.index.groupby(subClasses).values()))
          classDf = classDf.loc[next(subClasses)]
          for clazz in subClasses:
            newXi.append(set(clazz))
      newXi.append(set(classDf.index.values))

    if newXi == prevXi:
      return organize(newXi, sm)
    prevXi = newXi

# Epsilon transitions removal

In [ ]:
cleanSM, closures = removeEpsilonTransitions(sm)
print(cleanSM)
print('\nClosures:', closures)

                                   a                   b
S00(q1)  {S01(q0), S00(q1), Sf0(qf)}           {S00(q1)}
S01(q0)  {S01(q0), S00(q1), Sf0(qf)}  {S00(q1), Sf0(qf)}
Sf0(qf)  {S01(q0), S00(q1), Sf0(qf)}           {S00(q1)}

Closures: {'q1': {'q1'}, 'qf': {'q1', 'qf'}, 'q0': {'q1', 'q0'}}


In [ ]:
print(f'S: {cleanSM.S}', f'S0: {cleanSM.S0}', f'Sf: {cleanSM.Sf}')

S: {'S01(q0)', 'S00(q1)', 'Sf0(qf)'} S0: {'S01(q0)', 'S00(q1)'} Sf: {'Sf0(qf)'}


# Determinization

In [ ]:
determined, mappings = determine(cleanSM)
print(determined)
print('\nMappings:', mappings)

       a     b
P0  {P2}  {P1}
P2  {P2}  {P1}
P1  {P2}  {P3}
P3  {P2}  {P3}

Mappings: {'P0': {'S01(q0)', 'S00(q1)'}, 'P1': {'S00(q1)', 'Sf0(qf)'}, 'P2': {'S01(q0)', 'Sf0(qf)', 'S00(q1)'}, 'P3': {'S00(q1)'}}


In [ ]:
print(f'S: {determined.S}', f'S0: {determined.S0}', f'Sf: {determined.Sf}')

S: {'P3', 'P1', 'P2', 'P0'} S0: {'P0'} Sf: {'P1', 'P2'}


# Check path

In [ ]:
print(f"Should be True: {determined.checkPath(['a', 'b', 'b', 'b', 'b', 'b', 'a'])}")
print(f"Should be False: {determined.checkPath(['a','b','b'])}")

Should be True: True
Should be False: False


# Minimization

In [ ]:
minimized, classes = minimize(sm)
print(minimized)
print('\nClasses:', classes)

       a    b
M0    M1   M1
M1    M1  Mf0
Mf0  Mf0  Mf0

Classes: {'Mf0': {'q4', 'q5', 'q3'}, 'M0': {'q0'}, 'M1': {'q2', 'q1'}}


In [ ]:
print(f'S: {minimized.S}', f'S0: {minimized.S0}', f'Sf: {minimized.Sf}')

S: {'M1', 'M0', 'Mf0'} S0: {'M0'} Sf: {'Mf0'}




---



---


# Grammar


---



---



In [ ]:
from typing import Set, Tuple, List, Iterable

In [ ]:
def trimDuplicate(left: Iterable, right: Iterable, until: int = None):
  l1, l2 = len(left), len(right)
  if until:
    default = min(until, l1, l2)
    zipped = zip(left[:until], right[:until])
  else:
    default = min(l1, l2)
    zipped = zip(left, right)
  idx = next(iter([i for i, (l, r) in enumerate(zipped) if l != r]), default)
  return left[idx:], right[idx:]

class Grammar:
  def __init__(self, T: Set[str], N: Set[str], P: List[Tuple[str, str]], S: str):
    self.T = T.copy()
    self.N = N.copy()
    if not all(token in self.N for token in self.tokenize_string(S)):
      raise ValueError("S should be in N.")
    self.P = P.copy()
    self.S = S
  
  def tokenize_string(self, s: str) -> List[str]:
    if s == '':
      return []
    tokens = []
    sub = s
    while sub:
      found_match = False
      for token in sorted(self.T.union(self.N), key=len, reverse=True):
        if sub.startswith(token):
          tokens.append(token)
          sub = sub[len(token):]
          found_match = True
          break
      if not found_match:
        raise ValueError(f"Cannot tokenize string: '{s}'. Error in the substring: '{sub}'")
    return tokens
  
  def tokenize(self, p: Tuple[str, str]) -> Tuple[List[str], List[str]]:
    if p not in self.P:
        raise ValueError(f"Unresolved production: {p}")
    (left, right) = p
    return (self.tokenize_string(left), self.tokenize_string(right))

  # Type 3 R: A → Bγ | γ, where γ ∈ T*; A, B ∈ N.
  #           A → γB | γ, where γ ∈ T*; A, B ∈ N.
  # (S → ε is possible, but S shouldn't be in any rhs).
  def _chomsky_t3(self, tokenized) -> bool:
    rhsContainsS = any(self.S in rhs for _, rhs in tokenized)
    for lhs, rhs in tokenized:
      if len(lhs) != 1 or len(rhs) > 2:
        return False
      if lhs[0] not in self.N:
        return False
      if rhs == []:
        if lhs[0] == self.S and rhsContainsS: # S → ε
          return False
        continue
      bad = not(rhs[0] in self.N and rhs[-1] in self.T) # A → Bγ
      if bad:
        bad = not(rhs[-1] in self.N and rhs[0] in self.T) # # A → γB
      if bad:
        bad = rhs[0] not in self.T # A → γ
      if bad:
        return False
    return True

  # Type 2 CF: A → β, where A ∈ N, β ∈ {T ∪ N}+.
  def _chomsky_t2(self, tokenized) -> bool:
    for lhs, rhs in tokenized:
      if lhs == []:
        return False
      if not(len(lhs) == 1 and lhs[0] in self.N and rhs != []):
        return False
    return True

  # Type 1_1 CS: αAβ → αγβ, where α, β ∈ {T ∪ N}∗; A ∈ N; γ ∈ {T ∪ N}+ (S → ε is possible, but S shouldn't be in any rhs).
  def _chomsky_t1_1(self, tokenized) -> bool:
    rhsContainsS = any(self.S in rhs for _, rhs in tokenized)
    for lhs, rhs in tokenized:
      if len(lhs) == len(rhs) == 1:
        if lhs[0] == self.S and rhs == [] and rhsContainsS:
          return False  
      lastN = next(iter([i for i, token in enumerate(lhs) if token in self.N]), None)
      if lastN == None:
        return False
      lhs_t, rhs_t = trimDuplicate(lhs, rhs, lastN)
      lhs_t, rhs_t = trimDuplicate(lhs_t[::-1], rhs_t[::-1])
      A, γ = lhs_t[::-1], rhs_t[::-1]

      if len(A) != 1 or len(γ) == 0:
        return False
    return True  

  # Type 1_2 NT: α → β, where α, β ∈ {T ∪ N}+; |α|⩽|β| (S → ε is possible, but S shouldn't be in any rhs).
  def _chomsky_t1_2(self, tokenized) -> bool:
    rhsContainsS = any(self.S in rhs for _, rhs in tokenized)
    for lhs, rhs in tokenized:
      if lhs == []:
        return False
      if lhs[0] == self.S and rhs == [] and rhsContainsS: # S → ε
        return False 
      if len(lhs) > len(rhs): # |α|⩽|β|
        return False  
    return True

  # Type 0 U: α → β, where α ∈ {T U N}+, containing at least 1 x ∈ N; β ∈ {T ∪ N}∗.
  def _chomsky_t0(self, tokenized) -> bool:
    for lhs, _ in tokenized:
      if not any(token in self.N for token in lhs):
        return False
    return True

  def chomsky(self) -> str:
    tokenized = [self.tokenize(p) for p in self.P]

    if self._chomsky_t3(tokenized):
      return "Type 3 (regular) grammar"
    if self._chomsky_t2(tokenized):
      return "Type 2 (context-free) grammar"
    if self._chomsky_t1_1(tokenized):
      return "Type 1 (сontext-sensitive) grammar"
    if self._chomsky_t1_2(tokenized):
      return "Type 1 (non-trimming) grammar"
    if self._chomsky_t0(tokenized):
      return "Type 0 (unrestricted) grammar"
    
    return "Grammar of undefined type"

  def toSM(self):
    tokenized = [self.tokenize(p) for p in self.P]
    if not self._chomsky_t3(tokenized):
      raise ValueError(f"Cannot convert non-regular grammar.")

    dump_state = 'N`'
    sm = SM(self.T.union(['ε']), self.N.union([self.S, dump_state]), set([self.S]), set([dump_state]))
    
    # A → γB, where γ ∈ T*; A, B ∈ N. ===> a transition from state A to state B labelled γ
    # A → γ, where γ ∈ T; A ∈ N. ===> a transition from state A to state Dump labelled γ
    for lhs, rhs in tokenized:
      if rhs == []:
        sm.δ.at[lhs[0], 'ε'] = {dump_state}
      elif rhs[-1] in self.N:
        signal = rhs[0] if len(rhs) == 2 else 'ε'
        sm.δ.at[lhs[0], signal].add(rhs[-1]) 
      elif rhs[0] in self.T:
        # If exists lhs -> rhs[0] N => add N to F
        hasCover = False
        for lhs_c, rhs_c in tokenized:
          if lhs == lhs_c and len(rhs_c) == 2 and rhs_c[0] == rhs[0] and rhs_c[-1] in self.N:
            sm.Sf.add(rhs_c[-1])
            hasCover = True
        if not hasCover:
          sm.δ.at[lhs[0], rhs[0]].add(dump_state) 
    return sm

  def __repr__(self) -> str:
       return f'Grammar <T: {self.T}, N: {self.N}, P: {self.P}, S: {self.S}>'

# Classification Tests

### Type 3

In [ ]:
g3_1 = Grammar(T={'a', 'b'}, N={'S'}, P=[('S', 'aS'), ('S', 'b')], S='S')
g3_2 = Grammar({"0", "1"}, {"S"}, [("S", "01"), ("S", "10"), ("S", "")], "S")
print(g3_1.chomsky())
print(g3_2.chomsky())

Type 3 (regular) grammar
Type 3 (regular) grammar


### Type 2

In [ ]:
g2_1 = Grammar({"a", "b", "c"}, {"S", "A", "B", "C"}, [("S", "ABC"), ("A", "aA"), ("B", "bC"), ("C", "c")], "S")
g2_2 = Grammar({"a", "b"}, {"S", "A", "B"}, [("S", "AB"), ("A", "aA"), ("B", "b")], "S")
print(g2_1.chomsky())
print(g2_2.chomsky())

Type 2 (context-free) grammar
Type 2 (context-free) grammar


### Type 1 non-trimming

In [ ]:
g1_11 = Grammar(T={"a", "b"}, N={"S", "A", "B"}, P=[("SA", "aSB"), ("SB", "BB"), ("aB", "ab"), ("bB", "bb")], S="SA")
g1_12 = Grammar({"a", "b"}, {"S", "A", "B"}, [("S", "AB"), ("A", "aA"), ("Bb", "bb")], "S")
print(g1_11.chomsky())
print(g1_12.chomsky())

Type 1 (non-trimming) grammar
Type 1 (non-trimming) grammar


### Type 1 context-sensitive

In [ ]:
g1_21 = Grammar({"a", "b", "c"}, {"S", "T"}, [("S", "aTb"), ("S", "ab"), ("aT", "aaTb"), ("aT", "ac")], "S")
g1_22 = Grammar({"a", "b", "c"}, {"S", "A", "B"}, [("S", "abc"), ("S", "A"), ("A", "aABc"), ("A", "abc"), ("cB", "cBc"), ("bB", "bb")], "S")
print(g1_21.chomsky())
print(g1_22.chomsky())

Type 1 (сontext-sensitive) grammar
Type 1 (сontext-sensitive) grammar


### Type 0

In [ ]:
g0_1 = Grammar(T={'a', 'b'}, N={'S'}, P=[('S', 'aSb'), ('S', '')], S='S')
g0_2 = Grammar(T={'0', '1'}, N={'S', 'A'}, P=[('S', 'A0A1'), ('A', '0A1'), ('A', '')], S='S')
print(g0_1.chomsky())
print(g0_2.chomsky())

Type 0 (unrestricted) grammar
Type 0 (unrestricted) grammar


### Type undefined

In [ ]:
gu = Grammar(T={'0', '1'}, N={'S', 'A', 'B'}, P=[('S', 'AB'), ('A', '0A1'), ('B', '0B1'), ('', '0')], S='S')
print(gu.chomsky())

Grammar of undefined type


### Classification subject

In [ ]:
gg = Grammar(T={'+', '-', 'a', 'b'}, N={'S', 'x'}, P=[('S', 'x+x-x'), ('x', 'ax'), ('x', 'bx')], S='S')
print(gg.chomsky())

Type 2 (context-free) grammar


# Building an DFA from Grammar

In [ ]:
g = Grammar(T={'0', '1', 'a', 'b', 'c'}, N={'E', 'A', 'B', 'C', 'D'}, P=[
    ('E', '0A'), ('E', ''),
    ('A', 'aB'), ('A', 'aD'), 
    ('B', 'bB'), ('B', '1C'), ('B', 'c'),
    ('D', 'aD'), ('D', '0C'), ('D', 'c'),
    ('C', '0C'), ('C', '1C'), ('C', 'c')
], S='E')
print(g.chomsky())

Type 3 (regular) grammar


In [ ]:
sm = g.toSM()
sm

      0    1       a    b     c     ε
A    {}   {}  {B, D}   {}    {}    {}
B    {}  {C}      {}  {B}  {N`}    {}
C   {C}  {C}      {}   {}  {N`}    {}
D   {C}   {}     {D}   {}  {N`}    {}
E   {A}   {}      {}   {}    {}  {N`}
N`   {}   {}      {}   {}    {}    {}

In [ ]:
print(f'S: {sm.S}', f'S0: {sm.S0}', f'Sf: {sm.Sf}')

S: {'B', 'E', 'N`', 'A', 'D', 'C'} S0: {'E'} Sf: {'N`'}


# Determinization of converted FA

In [ ]:
determined, mappings = determine(sm)
print(determined)
print('\nMappings:', mappings)

       0     1     a     b     c
P0  {P1}    {}    {}    {}    {}
P1    {}    {}  {P2}    {}    {}
P2  {P4}  {P4}  {P5}  {P3}  {P6}
P3    {}  {P4}    {}  {P3}  {P6}
P4  {P4}  {P4}    {}    {}  {P6}
P5  {P4}    {}  {P5}    {}  {P6}
P6    {}    {}    {}    {}    {}

Mappings: {'P0': {'S00(E)', 'S01(N`)'}, 'P1': {'S2(A)'}, 'P2': {'S1(B)', 'S3(D)'}, 'P3': {'S1(B)'}, 'P4': {'S4(C)'}, 'P5': {'S3(D)'}, 'P6': {'S01(N`)'}}


In [ ]:
print(f'S: {determined.S}', f'S0: {determined.S0}', f'Sf: {determined.Sf}')

S: {'P4', 'P2', 'P0', 'P1', 'P5', 'P6', 'P3'} S0: {'P0'} Sf: {'P6', 'P0'}
